# Question 2 - VaR Calculations (Linear and Quadratic Approximations)

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st

## Initialize Variables

In [11]:
#Call option
So1= 50         #Intial Value for stock 1
K1 = 51         #Strike Price 
sig1 = 0.28     #Volatility 
T1 = 9/12       #Time to Maturity

#Put Option
So2= 20         #Intial Value for stock 1
K2 = 19         #Strike Price 
sig2 = 0.25     #Volatility 
T2 = 1          #Time to Maturity 

r = 0.06        #Risk free rate
rho = 0.4       #Correlation
N =1000#00       #Simulations 


## Main Code Block 

### Part a - Linear Approximation(Delta Approach) 

In [21]:
#Delta for call
d11 = (np.log(So1/K1) + (r+sig1**2/2)*T1)/(sig1*T1**0.5) 
deltac = st.norm.cdf(d11)

#Delta for put
d12 = (np.log(So2/K2) + (r+sig2**2/2)*T2)/(sig2*T2**0.5) 
deltap = st.norm.cdf(d12)-1

#Random variable generation
z1 = np.random.normal(0,1,N)
z2 = rho * z1 + (1-rho**2)**0.5 * np.random.normal(0,1,N)

#Changes in stock prices
delS1 = So1 * sig1 * z1 / (252**0.5)
delS2 = So2 * sig2 * z2 / (252**0.5)

#Changes in Portfolio 
delV = delS1*deltac + delS2*deltap

print("99% 10 day VaR for portfolio = ",np.quantile(delV,0.01)*(10**0.5))

99% 10 day VaR for portfolio =  -3.7665144652770692


### Part b - Quadratic Approximation (Delta Gamma Approach)

In [22]:
#Delta and Gamma for call
d11 = (np.log(So1/K1) + (r+sig1**2/2)*T1)/(sig1*T1**0.5) 
deltac = st.norm.cdf(d11)
Gammac = st.norm.pdf(d11)/(So1*sig1*(T1**0.5))

#Delta and Gamma for put
d12 = (np.log(So2/K2) + (r+sig2**2/2)*T2)/(sig2*T2**0.5) 
deltap = st.norm.cdf(d12)-1
Gammac = st.norm.pdf(d12)/(So1*sig2*(T2**0.5))

